# Validation of Remotely Sensed Droughts
**Comparing Spatial Trends**


## Overview

In this notebook we continue with drought indicators, as introduced in notebook1. The h SAF ASCAT drought indicator is based on soil moisture (temporal) anomalies, which require a long-term mean for a robust measure. The anomalies are than calculated as standardized deviations from the long-term mean.


AS explained in the first notebook, ASCAT SSM comes in units degree of saturation. Although spatial patterns,... , relative changes as used for drought anomaly detection

## Imports

In [ ]:
import cartopy.crs as ccrs
import datashader as ds
import holoviews as hv
import hvplot.pandas  # noqa
import numpy as np
import pandas as pd

## Standardized Precipitation-Evapotranspiration Index

Historically, many studies on drought monitoring have used hydro-meteorological indicators such as the Standardized Precipitation Index (SPI) and the more comprehensive Standardized Precipitation-Evapotranspiration Index (SPEI). However, satellite-based soil moisture estimates and linked drought anomaly indices might be more suitable for agricultural drought monitoring as they focus more directly on plant water requirements.

Despite this, SPEI indirectly accounts for soil conditions by combining both precipitation and potential evapotranspiration (PET), which measures atmospheric demand for water. Both SPEI and standardized soil moisture drought anomalies, such as Z scores, allow for comparisons across different regions and time scales.

To summarize:

- SPEI is a hydro-meteorological indicator for drought monitoring and thus indirectly measures soil moisture.
- Z scores are based on standardized anomalies derived from spaceborne (H SAF ASCAT) microwave-retrieved soil moisture data for the upper ~5 cm.

The purpose of this notebook is to compare both drought indicators, particularly focusing on the spatial extent of drought-affected areas. We will relate these indicators to recorded drought events in Mozambique between 2007 and 2021 ([The International Disaster Database](https://www.emdat.be/)).

| **Year/Period**       | **Affected Provinces**                                     | **Estimated Affected People**              |
|-----------------------|-------------------------------------------------------------|---------------------------------------------|
| **2008 (Early)**      | Maputo, Gaza, Inhambane, Manica, Sofala, Tete               | ~1.2 million         |
| **2010**              | Maputo, Gaza, Inhambane                                     | ~460,000             |
| **2015–2016 (El Niño)**| Maputo, Gaza, Inhambane, Sofala, Tete                      | ~2.3 million         |
| **2021**              | Cabo Delgado, Tete (south), Manica (north)                  | ~1.56 million        |


We first load the SPEI data in the following code cell.

In [ ]:
%run ./src/download_path.py

url = make_url("spei-6_25_monthly.csv")  # noqa
df1 = pd.read_csv(
    url,
    index_col=["time", "location_id"],
    parse_dates=["time"],
)
df1

Now let's also load the H SAF ASCAT surface soil moisture data which we used in notebook 1, but we will filter the data to have a comparable time range (up to the end of 2021) as we have for the SPEI data.

In [ ]:
url = make_url("ascat-6_25_ssm_monthly.csv")  # noqa
df2 = pd.read_csv(
    url,
    index_col=["time", "location_id"],
    parse_dates=["time"],
)[["zscore"]]
df2 = df2[df2.index <= df1.index.max()]
df2

## Merging Drought Indicator DATA

Now we will have to combine both datasets. Fortunately, both datasets are already projected on the same grid, so this task does not involve any reprojection and resampling of the data. We can combine the data with a SQL-style left join. A left join in pandas merges two DataFrames while keeping all rows from the left DataFrame and adding matching rows from the right DataFrame. If there's no match, it fills with NaN (see the figure for a schematic overview).

![Source: Ppandas User Guide](https://pandas.pydata.org/docs/user_guide)](https://pandas.pydata.org/docs/_images/merging_concat_axis1_join_axes.png)

The join operation is made easy here as we have already defined the same indexes in both datasets.

## 

merge() performs join operations similar to relational databases like SQL. Users who are familiar with SQL but new to pandas can reference a 

We use here the default operation which is "left join". THis is a type of join used to combine rows from two tables based on a related column between them. It returns all rows from the left table and includes matched rows from the right table. If there is no match, the result is `np.nan` for columns from the right table. Since we assigned indexes, where time and location_id define a unique observation, the join operations is based on this `pandas.MultIndex`. In other words, the left join ensures that all rows from the left table are included in the result set, even if there are no corresponding rows in the right table.

In [ ]:
df_wide = df1.join(df2)
df_wide

This operation produced a “record” or “wide” format dataset, typically there is one row for each subject, in this case the indicators "spei" and "zscores" as well as the coordinates. 

## Simplifying Drought Severity with Data Binning

We will convert the numeric data of the drought indicators `"spei"` and `"zscore"` into discrete categories using the pandas `cut` method. In pandas, binning data—also known as discretization or quantization—involves dividing continuous numerical data into discrete bins or intervals. This process is beneficial for simplifying data, managing outliers, creating histograms, and preparing data for machine learning algorithms that require categorical input. Additionally, we will label the binned data, thereby transforming the columns into pandas categorical data types. Pandas categorical data types are used to represent data that takes on a limited and usually fixed number of possible values (categories or classes). This type is particularly useful for categorical variables such as gender, days of the week, or survey responses. It offers efficient storage and operations for categorical data, including handling category ordering and missing values.

The process of binning and labeling drought data based on intensity is somewhat subjective, as the bin thresholds are often arbitrarily assigned and subject to debate. Here, we adhere to the guidelines provided by the World Meteorological Organization and the definitions by McKee et al. (1993)[^1] for standardized Soil Moisture (SM) based drought indices, where a "moderate" drought is defined as starting at -1 unit of standard deviations.

[^1]: 

In [ ]:
drought_labels = np.array(["extreme", "severe", "moderate", "mild", "normal"])
zscore_thresholds = [df_wide["zscore"].min(), -2, -1.5, -1, 0, df_wide["zscore"].max()]
spei_thresholds = [df_wide["spei"].min(), -2, -1.5, -1, 0, df_wide["spei"].max()]

Now we can use the labels and thresholds to bind the columns of thew drouhgt indicators. We make a copy of the original data to preserve.

In [ ]:
df_wide_cat = df_wide.copy()
df_wide_cat["zscore"] = pd.cut(df_wide.zscore, zscore_thresholds, labels=drought_labels)
df_wide_cat["spei"] = pd.cut(df_wide.spei, spei_thresholds, labels=drought_labels)
df_wide_cat

The simplified labelled drought indicators will now enables us a first step to assessing the spatial/areal extent.

For a sanity check of our results we will recreate our plot from notebook 1 but now with categorical data types for the drought indicators. We however need to one thing before we can plot this data. We need to reshape the data in a "long" or "stacked" format for `hvplot`. We can do this with pandas `melt`, where we declare that we keep the indexes as well as the coordinates as they are, but the indicator values are stacked on top of each other, where the variable column indentifies the drought indictor.


![Source: Ppandas User Guide](https://pandas.pydata.org/docs/user_guide)](https://pandas.pydata.org/docs/_images/reshaping_melt.png)

In [ ]:
df_long = df_wide_cat.melt(id_vars=["latitude", "longitude"], ignore_index=False)
df_long

In [ ]:
df_long.hvplot.points(
    x="longitude",
    y="latitude",
    c="value",
    groupby=["variable", "time"],
    x_sampling=0.1,
    y_sampling=0.1,
    aggregator=ds.by("value", ds.any()),
    rasterize=True,
    crs=ccrs.PlateCarree(),
    tiles=True,
    frame_width=500,
    clabel="Drought anomaly",
    cmap={
        "extreme": "#bb0c0c",
        "severe": "#c57b19",
        "moderate": "#b1bb29",
        "mild": "#1cd87a",
        "normal": "#ffffff",
    },
    colorbar=False,
)

## Spatial Extent

Now that we simplified the drought intensity to classes, let's turn to calculating the spatial extend of droughts through time. This way we can see how both standardized drought indexes compare to each other. For this we can conveniently use the pandas `value_count` method on the new categorical columns `"spei"` and `"zscore"`.

In [ ]:
col_spei = df_wide_cat.groupby(level=0)["spei"].value_counts(normalize=True).unstack()

In [ ]:
col_zscore = (
    df_wide_cat.groupby(level=0)["zscore"].value_counts(normalize=True).unstack()
)

We combine these results

In [ ]:
new_keys = pd.Index(["spei", "zscore"], name="indicator")
df_drought_extend = pd.concat(
    [col_spei, col_zscore],
    keys=new_keys,
)
df_drought_extend

In [ ]:
mozambique_droughts = [
    {"time": "2008-01-01", "people_affected": 1.02},
    {"time": "2010-01-01", "people_affected": 0.46},
    {"time": "2016-01-01", "people_affected": 2.30},
    {"time": "2021-01-01", "people_affected": 1.56},
]

df_droughts = pd.DataFrame(mozambique_droughts).assign(y=1)
df_droughts["time"] = pd.to_datetime(df_droughts["time"], format="%Y-%M-%d")
df_droughts.set_index("time", inplace=True)
labels = df_droughts.hvplot.labels(
    x="time",
    y="y",
    text="{people_affected} mill. people",
    text_baseline="bottom_left",
    hover=False,
    angle=85,
    text_font_size="14px",
)
offset = hv.dim("y") - 0.1
points = df_droughts.hvplot.points(
    x="time", y="y", color="black", hover=False, transforms={"y": offset}
)
df_drought_extend.hvplot.area(
    x="time",
    y=drought_labels[::-1][2:],
    groupby="indicator",
    hover=False,
    frame_width=800,
    padding=((0.1, 0.1), (0, 0.9)),
) * labels * points

We see large differences for both drought indicators

In [ ]:
url = make_url("drought_indices-6_25_monthly.csv")  # noqa
df_drought_indices = pd.read_csv(
    url,
    index_col=["time", "location_id"],
    parse_dates=["time"],
)
df_drought_indices

In [ ]:
def calc_drought_areal_extend(df):
    # make drought categories
    col_names = df.drop(columns=["longitude", "latitude"]).columns
    for name in col_names:
        min_border = df[name].min()
        max_border = df[name].max()
        thresholds = np.array(
            [
                min_border if min_border < -2 else -2.1,
                -2,
                -1.5,
                -1,
                0,
                max_border if max_border > 0 else 0.1,
            ]
        )
        df[name] = pd.cut(df[name], thresholds, labels=drought_labels)

    # calculate relative extend of drought
    new_df = pd.concat(
        [
            df.groupby(level=0)[col].value_counts(normalize=True).unstack()
            for col in col_names
        ],
        keys=pd.Index(col_names, name="indicator"),
    )
    return new_df


df_drought_extend = calc_drought_areal_extend(df_drought_indices.copy())
df_drought_extend

In [ ]:
df_drought_extend.hvplot.area(
    x="time",
    y=drought_labels[::-1][2:],
    groupby="indicator",
    hover=False,
    frame_width=800,
    padding=((0.1, 0.1), (0, 0.9)),
) * labels * points

In [ ]:
df_long.hvplot.points(
    x="longitude",
    y="latitude",
    groupby=["variable", "time"],
    x_sampling=0.1,
    y_sampling=0.1,
    rasterize=True,
    aggregator=ds.count_cat("value"),
    datashade=True,
    crs=ccrs.PlateCarree(),
    tiles=True,
    frame_width=500,
    clabel="Drought anomaly",
    cmap={
        "Extreme": "#bb0c0c",
        "Severe": "#c57b19",
        "moderate": "#b1bb29",
        "mild": "#1cd87a",
        "normal": "#ffffff",
    },
)

In [ ]:
df_confusion = pd.crosstab(df_wide_cat["spei"], df_wide_cat["zscore"], dropna=False)
df_confusion

In [ ]:
tot_drought = df_confusion.loc["drought", :].sum()
sensitivity = df_confusion.loc["drought", "drought"] / tot_drought
sensitivity

In [ ]:
tot_no_drought = df_confusion.loc["no-drought", :].sum()
specificity = df_confusion.loc["no-drought", "no-drought"] / tot_no_drought
specificity

In [ ]:
balanced_accuracy = (sensitivity + specificity) / 2
balanced_accuracy